In [1]:
import json
import jieba
import re
import numpy as np
import pandas as pd
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [2]:
# 这是用来生成lsa向量的训练集，必须用这个数据集来拼接lsa向量
gen_data_lsa_cvjd_6108 = pd.read_pickle('../generalization_data/gen_data_lsa_cvjd_6108.pkl')
gen_data_lsa_cvjd_6108

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1,da_label,rank_label,resume_type,order_month
90,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,JD20220802164652385655,电商运营专家,NaN,130106,商家运营,8a69c47b82590db901825d9613d75194,杭州康晟健康管理咨询有限公司,1,...,19.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""以岭药业集团"", ""department"": ""电商事...",NaN,20220831,0,0,5.0,2022-08
183,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,JD20220520123312243433,java开发工程师（不限方向）,NaN,100101,Java,2c92071580b3a1010180dcf9c08d56cc,上海泛微软件有限公司,30,...,8.0,NaN,"['{""degree"": ""大专"", ""endDate"": ""2012-01"", ""id"":...","['{""companyName"": ""北京汉克时代科技有限公司"", ""description...","['{""description"": ""聘 勾招聘\\n源 于拉勾招 源于拉\\n简•历来\\...",20220831,1,1,4.0,2022-08
312,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,JD20220711151647286483,系统架构师,NaN,100704,架构师,2c92071580b3a1010180cc87f22b4dc9,西藏东财基金管理有限公司,1,...,9.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2012-06"", ""...","['{""endDate"": ""2021-08"", ""companyName"": ""锦泓集团""...","['{""endDate"": ""2022-08"", ""jobTitle"": ""技术经理&架构师...",20220831,2,2,4.0,2022-08
313,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,JD20220804100450540684,课程顾问,NaN,160000,销售,2c920715800e886301804aae0eb67a50,北京鑫源速达科技有限公司,10,...,1.0,NaN,"['{""endDate"": ""2022-06"", ""unified"": true, ""deg...","['{""endDate"": ""2021-08"", ""companyName"": ""上海创维集...","['{""name"": null}']",20220831,1,1,4.0,2022-08
314,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,JD20220216094710092035,课程顾问,NaN,140311,销售顾问,2c9207157ef7ed16017f0032a4c375ef,上海之合网络科技有限公司,40,...,1.0,NaN,"['{""endDate"": ""2022-06"", ""unified"": true, ""deg...","['{""endDate"": ""2021-08"", ""companyName"": ""上海创维集...","['{""name"": null}']",20220831,2,2,4.0,2022-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,JD20220822112125679712,品牌负责人,NaN,130402,运营总监,8a69c4df82b081d30182c37f39d00627,浙江温家味食品科技有限公司,1,...,11.0,NaN,"['{""endDate"": ""2012-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-06"", ""companyName"": ""江苏领迅食...","['{""name"": null}']",20220831,3,3,4.0,2022-08
82329,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,JD20220818193847553940,大区销售总监,NaN,140402,销售总监,8a69c47b82b07fdc0182b0b5e27a6357,上海普适导航科技股份有限公司,5,...,14.0,NaN,"['{""endDate"": ""2008-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-08"", ""companyName"": ""安徽庆宇光...","['{""name"": null}']",20220831,3,3,4.0,2022-08
82330,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,JD20220520123312243433,java开发工程师（不限方向）,NaN,100101,Java,2c92071580b3a1010180dcf9c08d56cc,上海泛微软件有限公司,30,...,5.0,NaN,"['{""endDate"": ""2018-07"", ""unified"": true, ""deg...","['{""endDate"": ""2022-08"", ""companyName"": ""泰猎人力(...","['{""description"": ""项目描述: 客服云智库是河南电信客服人员内部使用的一套...",20220831,1,1,4.0,2022-08
82331,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,JD20220629161617600853,Legal Senior Manager,NaN,150506,法务经理,2c9207157f347800017f8d372b237e20,曼恩商用车辆贸易（中国）有限公司,1,...,16.0,NaN,"['{""endDate"": ""2007-06"", ""unified"": true, ""deg...","['{""endDate"": ""2021-07"", ""companyName"": ""长城汽车""...","['{""name"": null}']",20220831,-1,2,4.0,2022-08


In [3]:
gen_data_lsa_cvjd_6108.reset_index(drop=True)

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1,da_label,rank_label,resume_type,order_month
0,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,JD20220802164652385655,电商运营专家,NaN,130106,商家运营,8a69c47b82590db901825d9613d75194,杭州康晟健康管理咨询有限公司,1,...,19.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""以岭药业集团"", ""department"": ""电商事...",NaN,20220831,0,0,5.0,2022-08
1,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,JD20220520123312243433,java开发工程师（不限方向）,NaN,100101,Java,2c92071580b3a1010180dcf9c08d56cc,上海泛微软件有限公司,30,...,8.0,NaN,"['{""degree"": ""大专"", ""endDate"": ""2012-01"", ""id"":...","['{""companyName"": ""北京汉克时代科技有限公司"", ""description...","['{""description"": ""聘 勾招聘\\n源 于拉勾招 源于拉\\n简•历来\\...",20220831,1,1,4.0,2022-08
2,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,JD20220711151647286483,系统架构师,NaN,100704,架构师,2c92071580b3a1010180cc87f22b4dc9,西藏东财基金管理有限公司,1,...,9.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2012-06"", ""...","['{""endDate"": ""2021-08"", ""companyName"": ""锦泓集团""...","['{""endDate"": ""2022-08"", ""jobTitle"": ""技术经理&架构师...",20220831,2,2,4.0,2022-08
3,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,JD20220804100450540684,课程顾问,NaN,160000,销售,2c920715800e886301804aae0eb67a50,北京鑫源速达科技有限公司,10,...,1.0,NaN,"['{""endDate"": ""2022-06"", ""unified"": true, ""deg...","['{""endDate"": ""2021-08"", ""companyName"": ""上海创维集...","['{""name"": null}']",20220831,1,1,4.0,2022-08
4,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,JD20220216094710092035,课程顾问,NaN,140311,销售顾问,2c9207157ef7ed16017f0032a4c375ef,上海之合网络科技有限公司,40,...,1.0,NaN,"['{""endDate"": ""2022-06"", ""unified"": true, ""deg...","['{""endDate"": ""2021-08"", ""companyName"": ""上海创维集...","['{""name"": null}']",20220831,2,2,4.0,2022-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6103,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,JD20220822112125679712,品牌负责人,NaN,130402,运营总监,8a69c4df82b081d30182c37f39d00627,浙江温家味食品科技有限公司,1,...,11.0,NaN,"['{""endDate"": ""2012-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-06"", ""companyName"": ""江苏领迅食...","['{""name"": null}']",20220831,3,3,4.0,2022-08
6104,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,JD20220818193847553940,大区销售总监,NaN,140402,销售总监,8a69c47b82b07fdc0182b0b5e27a6357,上海普适导航科技股份有限公司,5,...,14.0,NaN,"['{""endDate"": ""2008-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-08"", ""companyName"": ""安徽庆宇光...","['{""name"": null}']",20220831,3,3,4.0,2022-08
6105,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,JD20220520123312243433,java开发工程师（不限方向）,NaN,100101,Java,2c92071580b3a1010180dcf9c08d56cc,上海泛微软件有限公司,30,...,5.0,NaN,"['{""endDate"": ""2018-07"", ""unified"": true, ""deg...","['{""endDate"": ""2022-08"", ""companyName"": ""泰猎人力(...","['{""description"": ""项目描述: 客服云智库是河南电信客服人员内部使用的一套...",20220831,1,1,4.0,2022-08
6106,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,JD20220629161617600853,Legal Senior Manager,NaN,150506,法务经理,2c9207157f347800017f8d372b237e20,曼恩商用车辆贸易（中国）有限公司,1,...,16.0,NaN,"['{""endDate"": ""2007-06"", ""unified"": true, ""deg...","['{""endDate"": ""2021-07"", ""companyName"": ""长城汽车""...","['{""name"": null}']",20220831,-1,2,4.0,2022-08


In [4]:
pd.read_pickle('../generalization_data/gen_description_tfidf_lsa.pkl')

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0.114425,0.053468,-0.110449,-0.000414,0.077922,0.052198,-0.065681,0.053433,-0.047084,0.052608,...,0.003370,-0.003603,-0.071144,-0.013497,0.011143,-0.001161,-0.001739,-0.019630,0.033437,-0.079705
1,0.450187,-0.286379,0.243543,0.124966,-0.083365,-0.248026,-0.028472,0.433519,0.157351,-0.215049,...,0.010478,-0.020256,0.021174,-0.029691,-0.021322,0.007466,-0.003959,-0.012820,-0.034586,-0.000488
2,0.117757,-0.011553,0.068931,-0.019368,0.034283,-0.017140,0.003966,-0.023411,-0.032501,0.046184,...,-0.012104,-0.007032,0.020202,0.066465,0.015784,0.015048,0.030359,0.003329,0.001796,-0.033600
3,0.167660,0.080103,-0.094919,0.186250,0.152330,-0.025803,0.169278,0.038104,0.055463,-0.085552,...,-0.029705,-0.011755,0.012089,0.003651,0.006682,-0.028695,0.005617,-0.021611,-0.006179,0.019756
4,0.165185,0.083476,-0.095790,0.191046,0.156115,-0.026455,0.172255,0.038230,0.058611,-0.090204,...,-0.026103,-0.008237,0.006852,0.002990,0.011548,-0.031001,0.009802,-0.014868,0.009643,0.012507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6103,0.256799,0.177310,-0.057122,-0.041347,0.066737,0.070796,-0.166106,0.032757,0.020973,-0.027775,...,-0.042560,0.019839,-0.066739,-0.065670,-0.038295,-0.030264,-0.011418,0.041912,0.056678,-0.030501
6104,0.250568,0.185586,-0.059813,0.268673,0.026521,-0.045776,-0.033466,0.107338,-0.018514,-0.000381,...,0.031739,0.002281,0.036906,0.039602,-0.065171,-0.031183,0.003653,0.051910,-0.025298,0.012252
6105,0.450187,-0.286379,0.243543,0.124966,-0.083365,-0.248026,-0.028472,0.433519,0.157351,-0.215049,...,0.010478,-0.020256,0.021174,-0.029691,-0.021322,0.007466,-0.003959,-0.012820,-0.034586,-0.000488
6106,0.002312,0.001275,0.002012,0.010614,0.001989,-0.002606,0.017346,0.020062,0.530427,0.447457,...,-0.004383,-0.001078,0.003673,0.002062,-0.000015,-0.000579,-0.002167,0.001440,0.001515,-0.000902


# tfidf_lsa

In [5]:
# 读取生成的5份tfidf_pca文件，并拼接
# 注意，不是join的时候，concat是通过index拼接的。所以需要将索引重置。
sample = gen_data_lsa_cvjd_6108.reset_index(drop=True)

# 需要将这个向量变成一个列表
def tfidf_lsa_merge_fun(series):
    return list(series)

tfidf_lsa_file = ['gen_description_tfidf_lsa.pkl', 'gen_position_tfidf_lsa.pkl', 'gen_requirement_tfidf_lsa.pkl', 'gen_skills_tfidf_lsa.pkl', 'gen_title_category_tags_tfidf_lsa.pkl']
for pkl_file in tfidf_lsa_file:
    tfidf_lsa = pd.read_pickle('../generalization_data/'+pkl_file)
    print(pkl_file)
    #print(tfidf_pca)
    col_name = pkl_file.strip('.pkl') # 把skills的s去掉了
    tfidf_lsa[col_name] = tfidf_lsa.apply(tfidf_lsa_merge_fun, axis=1)
    sample = pd.concat([sample, tfidf_lsa[[col_name]]], axis=1)

tfidf_lsa_col_names = [pkl_file.strip('.pkl') for pkl_file in tfidf_lsa_file]
sample[['cv_id', 'jd_id']+tfidf_lsa_col_names]

gen_description_tfidf_lsa.pkl
gen_position_tfidf_lsa.pkl
gen_requirement_tfidf_lsa.pkl
gen_skills_tfidf_lsa.pkl
gen_title_category_tags_tfidf_lsa.pkl


,cv_id,jd_id,gen_description_tfidf_lsa,gen_position_tfidf_lsa,gen_requirement_tfidf_lsa,gen_skills_tfidf_lsa,gen_title_category_tags_tfidf_lsa
0,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,"[0.11442535037227546, 0.0534682035829645, -0.1...","[0.01410938306399229, -0.0013842289481747908, ...","[0.16147995305672858, -0.07191881877515002, -0...","[0.2948113481706559, -0.1819010648461708, 0.01...","[0.4002958143776276, -0.18420725834968477, -0...."
1,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,"[0.4501867000452721, -0.2863792718427415, 0.24...","[0.1833094661547266, 0.5715641770033284, -0.00...","[0.42336579128110136, 0.4415293631473783, 0.08...","[0.3123019409501142, 0.25026041462341286, -0.0...","[0.2136262495353285, 0.6889780980076565, -0.03..."
2,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,"[0.11775671151124475, -0.011553286324814227, 0...","[0.1242462692048715, 0.059776515919823886, 0.0...","[0.13464711322627812, 0.03601774564762096, 0.0...","[0.2592719578121452, 0.19863144276159853, -0.1...","[0.0846159262961945, 0.14089215219884776, -0.0..."
3,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,"[0.16766031367661063, 0.08010325682707635, -0....","[0.040317751879249734, -0.004257781078691267, ...","[0.2017619187252856, -0.1156944482993234, 0.12...","[0.20906464927123727, -0.16293904571516468, 0....","[0.10444744556880664, -0.03406922047311521, -0..."
4,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,"[0.1651847923619087, 0.08347563503643975, -0.0...","[0.040317751879249734, -0.004257781078691267, ...","[0.19550223541803413, -0.1128925577464503, 0.1...","[0.20906464927123727, -0.16293904571516468, 0....","[0.11715741531242865, -0.03963572163058257, -0..."
...,...,...,...,...,...,...,...
6103,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,"[0.2567986184645047, 0.1773098969033448, -0.05...","[0.09276561511018605, -0.009011915759238062, 0...","[0.3645132923299154, -0.22668855509127542, 0.0...","[0.44498980088785645, -0.21377671142306073, -0...","[0.2028996048643067, -0.07039791359665602, -0...."
6104,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,"[0.25056830107660755, 0.1855862403422956, -0.0...","[0.22000723570438951, -0.03295513889781696, 0....","[0.1900283816382599, -0.101841367221085, 0.037...","[0.1849982062512915, -0.038574753114996224, -0...","[0.2731820444500037, -0.09003645686019562, -0...."
6105,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,"[0.4501867000452721, -0.2863792718427415, 0.24...","[0.153610738681069, 0.7683157290699305, -0.012...","[0.42336579128110136, 0.4415293631473783, 0.08...","[0.1645560370825661, 0.2974652276594942, -0.06...","[0.2136262495353285, 0.6889780980076565, -0.03..."
6106,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,"[0.0023119905200976844, 0.0012746774424303585,...","[0.06146393099923767, -0.005610505861232773, 0...","[0.002297610145839555, -0.0017582815189133496,...","[0.21403696108733744, -0.09024431543561566, -0...","[0.05431574129285235, -0.0031620523933319758, ..."


In [6]:
sample[['cv_id', 'jd_id']+tfidf_lsa_col_names].to_pickle('../generalization_data/gen_data_lsa_cvjd_merge_6108.pkl')

In [7]:
cvjd_lsa = sample[['cv_id', 'jd_id']+tfidf_lsa_col_names]
cvjd_lsa

,cv_id,jd_id,gen_description_tfidf_lsa,gen_position_tfidf_lsa,gen_requirement_tfidf_lsa,gen_skills_tfidf_lsa,gen_title_category_tags_tfidf_lsa
0,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,"[0.11442535037227546, 0.0534682035829645, -0.1...","[0.01410938306399229, -0.0013842289481747908, ...","[0.16147995305672858, -0.07191881877515002, -0...","[0.2948113481706559, -0.1819010648461708, 0.01...","[0.4002958143776276, -0.18420725834968477, -0...."
1,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,"[0.4501867000452721, -0.2863792718427415, 0.24...","[0.1833094661547266, 0.5715641770033284, -0.00...","[0.42336579128110136, 0.4415293631473783, 0.08...","[0.3123019409501142, 0.25026041462341286, -0.0...","[0.2136262495353285, 0.6889780980076565, -0.03..."
2,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,"[0.11775671151124475, -0.011553286324814227, 0...","[0.1242462692048715, 0.059776515919823886, 0.0...","[0.13464711322627812, 0.03601774564762096, 0.0...","[0.2592719578121452, 0.19863144276159853, -0.1...","[0.0846159262961945, 0.14089215219884776, -0.0..."
3,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,"[0.16766031367661063, 0.08010325682707635, -0....","[0.040317751879249734, -0.004257781078691267, ...","[0.2017619187252856, -0.1156944482993234, 0.12...","[0.20906464927123727, -0.16293904571516468, 0....","[0.10444744556880664, -0.03406922047311521, -0..."
4,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,"[0.1651847923619087, 0.08347563503643975, -0.0...","[0.040317751879249734, -0.004257781078691267, ...","[0.19550223541803413, -0.1128925577464503, 0.1...","[0.20906464927123727, -0.16293904571516468, 0....","[0.11715741531242865, -0.03963572163058257, -0..."
...,...,...,...,...,...,...,...
6103,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,"[0.2567986184645047, 0.1773098969033448, -0.05...","[0.09276561511018605, -0.009011915759238062, 0...","[0.3645132923299154, -0.22668855509127542, 0.0...","[0.44498980088785645, -0.21377671142306073, -0...","[0.2028996048643067, -0.07039791359665602, -0...."
6104,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,"[0.25056830107660755, 0.1855862403422956, -0.0...","[0.22000723570438951, -0.03295513889781696, 0....","[0.1900283816382599, -0.101841367221085, 0.037...","[0.1849982062512915, -0.038574753114996224, -0...","[0.2731820444500037, -0.09003645686019562, -0...."
6105,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,"[0.4501867000452721, -0.2863792718427415, 0.24...","[0.153610738681069, 0.7683157290699305, -0.012...","[0.42336579128110136, 0.4415293631473783, 0.08...","[0.1645560370825661, 0.2974652276594942, -0.06...","[0.2136262495353285, 0.6889780980076565, -0.03..."
6106,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,"[0.0023119905200976844, 0.0012746774424303585,...","[0.06146393099923767, -0.005610505861232773, 0...","[0.002297610145839555, -0.0017582815189133496,...","[0.21403696108733744, -0.09024431543561566, -0...","[0.05431574129285235, -0.0031620523933319758, ..."


# work_duration equal_job salary

In [8]:
work_duration_mean_df = pd.read_pickle('../generalization_data/gen_work_duration_mean.pkl')
equal_job_df = pd.read_pickle('../generalization_data/gen_equal_job.pkl')
salary_df = pd.read_pickle('../generalization_data/gen_salary.pkl')

In [9]:
cvjd_lsa_work_duration_equal_job_salary = cvjd_lsa.\
    join(work_duration_mean_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner').\
    join(equal_job_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner').\
    join(salary_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')

cvjd_lsa_work_duration_equal_job_salary

,cv_id,jd_id,gen_description_tfidf_lsa,gen_position_tfidf_lsa,gen_requirement_tfidf_lsa,gen_skills_tfidf_lsa,gen_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
0,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,"[0.11442535037227546, 0.0534682035829645, -0.1...","[0.01410938306399229, -0.0013842289481747908, ...","[0.16147995305672858, -0.07191881877515002, -0...","[0.2948113481706559, -0.1819010648461708, 0.01...","[0.4002958143776276, -0.18420725834968477, -0....",1122.000000,3,-1,800
1,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,"[0.4501867000452721, -0.2863792718427415, 0.24...","[0.1833094661547266, 0.5715641770033284, -0.00...","[0.42336579128110136, 0.4415293631473783, 0.08...","[0.3123019409501142, 0.25026041462341286, -0.0...","[0.2136262495353285, 0.6889780980076565, -0.03...",718.800000,5,-1,-1
2,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,"[0.11775671151124475, -0.011553286324814227, 0...","[0.1242462692048715, 0.059776515919823886, 0.0...","[0.13464711322627812, 0.03601774564762096, 0.0...","[0.2592719578121452, 0.19863144276159853, -0.1...","[0.0846159262961945, 0.14089215219884776, -0.0...",704.714286,2,-1,528
3,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,"[0.16766031367661063, 0.08010325682707635, -0....","[0.040317751879249734, -0.004257781078691267, ...","[0.2017619187252856, -0.1156944482993234, 0.12...","[0.20906464927123727, -0.16293904571516468, 0....","[0.10444744556880664, -0.03406922047311521, -0...",46.000000,4,108,72
4,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,"[0.1651847923619087, 0.08347563503643975, -0.0...","[0.040317751879249734, -0.004257781078691267, ...","[0.19550223541803413, -0.1128925577464503, 0.1...","[0.20906464927123727, -0.16293904571516468, 0....","[0.11715741531242865, -0.03963572163058257, -0...",46.000000,3,108,72
...,...,...,...,...,...,...,...,...,...,...,...
6103,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,"[0.2567986184645047, 0.1773098969033448, -0.05...","[0.09276561511018605, -0.009011915759238062, 0...","[0.3645132923299154, -0.22668855509127542, 0.0...","[0.44498980088785645, -0.21377671142306073, -0...","[0.2028996048643067, -0.07039791359665602, -0....",1268.000000,4,-1,-1
6104,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,"[0.25056830107660755, 0.1855862403422956, -0.0...","[0.22000723570438951, -0.03295513889781696, 0....","[0.1900283816382599, -0.101841367221085, 0.037...","[0.1849982062512915, -0.038574753114996224, -0...","[0.2731820444500037, -0.09003645686019562, -0....",1583.333333,3,120,120
6105,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,"[0.4501867000452721, -0.2863792718427415, 0.24...","[0.153610738681069, 0.7683157290699305, -0.012...","[0.42336579128110136, 0.4415293631473783, 0.08...","[0.1645560370825661, 0.2974652276594942, -0.06...","[0.2136262495353285, 0.6889780980076565, -0.03...",487.000000,5,300,216
6106,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,"[0.0023119905200976844, 0.0012746774424303585,...","[0.06146393099923767, -0.005610505861232773, 0...","[0.002297610145839555, -0.0017582815189133496,...","[0.21403696108733744, -0.09024431543561566, -0...","[0.05431574129285235, -0.0031620523933319758, ...",968.666667,1,900,900


# new feature

In [10]:
# 保存为new feature
cvjd_lsa_work_duration_equal_job_salary.to_pickle('../generalization_data/gen_new_feature_6108.pkl')


# raw feature

In [11]:
# 这是用来生成lsa向量的训练集，必须用这个数据集来拼接lsa向量
# 这份文件其实是用raw_cvjd的数据和train id进行join得到的
gen_data_lsa_cvjd_6108 = pd.read_pickle('../generalization_data/gen_data_lsa_cvjd_6108.pkl')
gen_data_lsa_cvjd_6108

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1,da_label,rank_label,resume_type,order_month
90,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,JD20220802164652385655,电商运营专家,NaN,130106,商家运营,8a69c47b82590db901825d9613d75194,杭州康晟健康管理咨询有限公司,1,...,19.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""以岭药业集团"", ""department"": ""电商事...",NaN,20220831,0,0,5.0,2022-08
183,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,JD20220520123312243433,java开发工程师（不限方向）,NaN,100101,Java,2c92071580b3a1010180dcf9c08d56cc,上海泛微软件有限公司,30,...,8.0,NaN,"['{""degree"": ""大专"", ""endDate"": ""2012-01"", ""id"":...","['{""companyName"": ""北京汉克时代科技有限公司"", ""description...","['{""description"": ""聘 勾招聘\\n源 于拉勾招 源于拉\\n简•历来\\...",20220831,1,1,4.0,2022-08
312,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,JD20220711151647286483,系统架构师,NaN,100704,架构师,2c92071580b3a1010180cc87f22b4dc9,西藏东财基金管理有限公司,1,...,9.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2012-06"", ""...","['{""endDate"": ""2021-08"", ""companyName"": ""锦泓集团""...","['{""endDate"": ""2022-08"", ""jobTitle"": ""技术经理&架构师...",20220831,2,2,4.0,2022-08
313,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,JD20220804100450540684,课程顾问,NaN,160000,销售,2c920715800e886301804aae0eb67a50,北京鑫源速达科技有限公司,10,...,1.0,NaN,"['{""endDate"": ""2022-06"", ""unified"": true, ""deg...","['{""endDate"": ""2021-08"", ""companyName"": ""上海创维集...","['{""name"": null}']",20220831,1,1,4.0,2022-08
314,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,JD20220216094710092035,课程顾问,NaN,140311,销售顾问,2c9207157ef7ed16017f0032a4c375ef,上海之合网络科技有限公司,40,...,1.0,NaN,"['{""endDate"": ""2022-06"", ""unified"": true, ""deg...","['{""endDate"": ""2021-08"", ""companyName"": ""上海创维集...","['{""name"": null}']",20220831,2,2,4.0,2022-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,JD20220822112125679712,品牌负责人,NaN,130402,运营总监,8a69c4df82b081d30182c37f39d00627,浙江温家味食品科技有限公司,1,...,11.0,NaN,"['{""endDate"": ""2012-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-06"", ""companyName"": ""江苏领迅食...","['{""name"": null}']",20220831,3,3,4.0,2022-08
82329,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,JD20220818193847553940,大区销售总监,NaN,140402,销售总监,8a69c47b82b07fdc0182b0b5e27a6357,上海普适导航科技股份有限公司,5,...,14.0,NaN,"['{""endDate"": ""2008-06"", ""unified"": true, ""deg...","['{""endDate"": ""2022-08"", ""companyName"": ""安徽庆宇光...","['{""name"": null}']",20220831,3,3,4.0,2022-08
82330,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,JD20220520123312243433,java开发工程师（不限方向）,NaN,100101,Java,2c92071580b3a1010180dcf9c08d56cc,上海泛微软件有限公司,30,...,5.0,NaN,"['{""endDate"": ""2018-07"", ""unified"": true, ""deg...","['{""endDate"": ""2022-08"", ""companyName"": ""泰猎人力(...","['{""description"": ""项目描述: 客服云智库是河南电信客服人员内部使用的一套...",20220831,1,1,4.0,2022-08
82331,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,JD20220629161617600853,Legal Senior Manager,NaN,150506,法务经理,2c9207157f347800017f8d372b237e20,曼恩商用车辆贸易（中国）有限公司,1,...,16.0,NaN,"['{""endDate"": ""2007-06"", ""unified"": true, ""deg...","['{""endDate"": ""2021-07"", ""companyName"": ""长城汽车""...","['{""name"": null}']",20220831,-1,2,4.0,2022-08


In [12]:
# 这些特征在原始的cv和jd表里
raw_columns = ['age', 'recruit_number', 'workYear', 'min_month_salary', 'max_month_salary', 'pay_months']

# 将raw特征保存
gen_data_lsa_cvjd_6108[['cv_id', 'jd_id'] + raw_columns].to_pickle('../generalization_data/gen_raw_feature_6108.pkl')


# ext特征

In [13]:
# 这些特征在sample pipeline的表里
ext_columns = ['school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv', 'location_range_jd', 'degree_index_cv', 'degree_index_jd', 'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv']


In [14]:
# 读取包含label的sample表
all_sample = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)

gen_path = '../generalization_data/cvjd_202208_6108.pkl'
gen_id = pd.read_pickle(gen_path)[['cv_id', 'jd_id', 'order_month']]

all_sample = all_sample.join(gen_id.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
all_sample

/Users/jinrirencai/opt/anaconda3/envs/tsinghua/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cv_id,jd_id,cv_code,jd_code,last_opt,da_label,rank_label,title,deadline,category_id,...,industry_jd2vec,industry_vec2,majorName_cut,majorName_jd2vec,majorName_vec2,skills_cut,skills_jd2vec,skills_vec2,ds,order_month
470,8a69c41582926ae60182a07b24c1048c,8a69f62f82926a8601829f1379a07648,C1008764509940281344,JD20220812101945714568,reject,2,2,多媒体算法工程师（视频算法）,NaN,100000,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['长春', '理工大学']","[0.3985162079334259, -0.9995342493057251, -0.0...","[0.11997296661138535, 0.26912960410118103, -0....","['Arm', ',', 'Git', ',', 'C++', ',', 'Demo', '...","[0.38910111784935, -0.09600549191236496, -0.16...","[0.03988659381866455, 0.09070292860269547, -0....",20220831,2022-08
471,8a69c41582926ae60182a0e09bf27664,8a69d77e82926b410182a083fbf602c0,C1008792400581292032,JD20220815155940455233,accept,3,3,采购工程师,NaN,250103,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['吉林大学'],"[0.09345181286334991, 0.009083555079996586, 0....","[0.23355810344219208, 0.3138801157474518, -0.1...","['Pc', ',', '篮球', ',', '行政', '管理']","[0.5843684673309326, 0.09454917907714844, 0.02...","[0.39833325147628784, 0.178728848695755, -0.74...",20220831,2022-08
472,8a69c41582926ae60182a119de713808,8a69f68782590ecb01826326b5e94919,C1008808140025819136,JD20220803180057511310,post,-1,2,销售工程师,NaN,210100,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['黑龙江', '大学']","[0.5287548899650574, -0.42875587940216064, 0.1...","[0.9195342659950256, 0.8215621709823608, -0.00...","['优化', ',', '架构', ',', '运营', ',', '搭建', ',', '...","[0.5232886075973511, 0.05269351974129677, -0.0...","[0.4764364957809448, -0.18185757100582123, -0....",20220831,2022-08
473,8a69c41582a3cc240182b0354c0e7bad,2c92071581ebf3390181ec9e500755e4,C1009871210596724736,JD20220711173647069409,drop,1,1,产品总监,NaN,110302,...,"[0.06698619574308395, -0.19367165863513947, -0...","[1.3965752124786377, 0.07573889195919037, -1.1...","['安徽', '财经大学']","[0.530929446220398, -0.6542360782623291, 0.157...","[0.24919170141220093, 0.20309053361415863, -0....","['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",20220831,2022-08
474,8a69c4a3827cc886018284b9612d0f03,8a69f601823a92b0018243dcb9c45f7a,C1006811292427849728,JD20220728161155132235,link,-1,1,GOOGLE推广,NaN,140104,...,"[0.3468320667743683, -0.4120320677757263, -0.0...","[0.7798252105712891, -0.039847563952207565, -0...",['湖南师范大学'],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","['Crm', ',', '电商', ',', '营销', ',', '策划', ',', ...","[0.5233139991760254, -0.056362878531217575, -0...","[0.47880709171295166, -0.09375546127557755, -0...",20220831,2022-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82316,8a69d71b82add4e10182aebff7d016d2,8a69c468826ee9b4018290a64a0c01bb,C1001546295099396096,JD20220812140313222488,reject,2,2,营销总监-服务中心,NaN,200101,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['中山大学'],"[0.0773172527551651, -0.2358638048171997, 0.00...","[-0.10793008655309677, 0.5528307557106018, 0.1...","['优化', ',', '策划', ',', '营销', ',', '团队', '建设', ...","[0.6373644471168518, 0.0508057177066803, 0.004...","[0.5669321417808533, -0.2156238704919815, -0.3...",20220831,2022-08
82325,8a69d7b4826d7dfb01826d7efad200f1,2c920715818a462a01818e3f9c0a2337,C972256921547575656,JD20220615174226328653,accept,3,3,Android-专家及leader以上,NaN,100202,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['University', ' ', 'of', ' ', 'Illinois']","[0.08156125992536545, 0.31228697299957275, 0.1...","[-0.15771661698818207, 0.5672192573547363, -0....","['Js', ',', 'C#', ',', 'Web', ',', 'CSS', ',',...","[0.3464146852493286, 0.07

In [15]:
gen_id.columns

Index(['cv_id', 'jd_id', 'order_month'], dtype='object')

In [16]:
# 将ext特征保存
all_sample[['cv_id', 'jd_id'] + ext_columns].to_pickle('../generalization_data/gen_ext_feature_6108.pkl')

# 拼接new raw ext特征

In [17]:
base_path = '../generalization_data/'
raw_feature_file = 'gen_raw_feature_6108.pkl'
ext_feature_file = 'gen_ext_feature_6108.pkl'
new_feature_file = 'gen_new_feature_6108.pkl'

# 读取特征文件
raw_feature = pd.read_pickle(base_path + raw_feature_file)
ext_feature = pd.read_pickle(base_path + ext_feature_file)
new_feature = pd.read_pickle(base_path + new_feature_file)


In [18]:
new_feature

,cv_id,jd_id,gen_description_tfidf_lsa,gen_position_tfidf_lsa,gen_requirement_tfidf_lsa,gen_skills_tfidf_lsa,gen_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
0,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,"[0.11442535037227546, 0.0534682035829645, -0.1...","[0.01410938306399229, -0.0013842289481747908, ...","[0.16147995305672858, -0.07191881877515002, -0...","[0.2948113481706559, -0.1819010648461708, 0.01...","[0.4002958143776276, -0.18420725834968477, -0....",1122.000000,3,-1,800
1,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,"[0.4501867000452721, -0.2863792718427415, 0.24...","[0.1833094661547266, 0.5715641770033284, -0.00...","[0.42336579128110136, 0.4415293631473783, 0.08...","[0.3123019409501142, 0.25026041462341286, -0.0...","[0.2136262495353285, 0.6889780980076565, -0.03...",718.800000,5,-1,-1
2,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,"[0.11775671151124475, -0.011553286324814227, 0...","[0.1242462692048715, 0.059776515919823886, 0.0...","[0.13464711322627812, 0.03601774564762096, 0.0...","[0.2592719578121452, 0.19863144276159853, -0.1...","[0.0846159262961945, 0.14089215219884776, -0.0...",704.714286,2,-1,528
3,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,"[0.16766031367661063, 0.08010325682707635, -0....","[0.040317751879249734, -0.004257781078691267, ...","[0.2017619187252856, -0.1156944482993234, 0.12...","[0.20906464927123727, -0.16293904571516468, 0....","[0.10444744556880664, -0.03406922047311521, -0...",46.000000,4,108,72
4,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,"[0.1651847923619087, 0.08347563503643975, -0.0...","[0.040317751879249734, -0.004257781078691267, ...","[0.19550223541803413, -0.1128925577464503, 0.1...","[0.20906464927123727, -0.16293904571516468, 0....","[0.11715741531242865, -0.03963572163058257, -0...",46.000000,3,108,72
...,...,...,...,...,...,...,...,...,...,...,...
6103,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,"[0.2567986184645047, 0.1773098969033448, -0.05...","[0.09276561511018605, -0.009011915759238062, 0...","[0.3645132923299154, -0.22668855509127542, 0.0...","[0.44498980088785645, -0.21377671142306073, -0...","[0.2028996048643067, -0.07039791359665602, -0....",1268.000000,4,-1,-1
6104,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,"[0.25056830107660755, 0.1855862403422956, -0.0...","[0.22000723570438951, -0.03295513889781696, 0....","[0.1900283816382599, -0.101841367221085, 0.037...","[0.1849982062512915, -0.038574753114996224, -0...","[0.2731820444500037, -0.09003645686019562, -0....",1583.333333,3,120,120
6105,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,"[0.4501867000452721, -0.2863792718427415, 0.24...","[0.153610738681069, 0.7683157290699305, -0.012...","[0.42336579128110136, 0.4415293631473783, 0.08...","[0.1645560370825661, 0.2974652276594942, -0.06...","[0.2136262495353285, 0.6889780980076565, -0.03...",487.000000,5,300,216
6106,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,"[0.0023119905200976844, 0.0012746774424303585,...","[0.06146393099923767, -0.005610505861232773, 0...","[0.002297610145839555, -0.0017582815189133496,...","[0.21403696108733744, -0.09024431543561566, -0...","[0.05431574129285235, -0.0031620523933319758, ...",968.666667,1,900,900


In [19]:
# 3份特征文件拼接
medium = raw_feature.join(ext_feature.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature = medium.join(new_feature.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,job_intentstatus_cv,gen_description_tfidf_lsa,gen_position_tfidf_lsa,gen_requirement_tfidf_lsa,gen_skills_tfidf_lsa,gen_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
90,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,41.0,1,19.0,60.0,70.0,12.0,"['211', '双一流']",[],...,3,"[0.11442535037227546, 0.0534682035829645, -0.1...","[0.01410938306399229, -0.0013842289481747908, ...","[0.16147995305672858, -0.07191881877515002, -0...","[0.2948113481706559, -0.1819010648461708, 0.01...","[0.4002958143776276, -0.18420725834968477, -0....",1122.000000,3,-1,800
183,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,33.0,30,8.0,15.0,30.0,14.0,[],[],...,3,"[0.4501867000452721, -0.2863792718427415, 0.24...","[0.1833094661547266, 0.5715641770033284, -0.00...","[0.42336579128110136, 0.4415293631473783, 0.08...","[0.3123019409501142, 0.25026041462341286, -0.0...","[0.2136262495353285, 0.6889780980076565, -0.03...",718.800000,5,-1,-1
312,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,33.0,1,9.0,30.0,45.0,16.0,"['211', '双一流']",['211'],...,2,"[0.11775671151124475, -0.011553286324814227, 0...","[0.1242462692048715, 0.059776515919823886, 0.0...","[0.13464711322627812, 0.03601774564762096, 0.0...","[0.2592719578121452, 0.19863144276159853, -0.1...","[0.0846159262961945, 0.14089215219884776, -0.0...",704.714286,2,-1,528
313,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,22.0,10,1.0,15.0,20.0,12.0,[],[],...,0,"[0.16766031367661063, 0.08010325682707635, -0....","[0.040317751879249734, -0.004257781078691267, ...","[0.2017619187252856, -0.1156944482993234, 0.12...","[0.20906464927123727, -0.16293904571516468, 0....","[0.10444744556880664, -0.03406922047311521, -0...",46.000000,4,108,72
314,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,22.0,40,1.0,10.0,15.0,12.0,[],[],...,0,"[0.1651847923619087, 0.08347563503643975, -0.0...","[0.040317751879249734, -0.004257781078691267, ...","[0.19550223541803413, -0.1128925577464503, 0.1...","[0.20906464927123727, -0.16293904571516468, 0....","[0.11715741531242865, -0.03963572163058257, -0...",46.000000,3,108,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,31.0,1,11.0,30.0,40.0,12.0,[],[],...,0,"[0.2567986184645047, 0.1773098969033448, -0.05...","[0.09276561511018605, -0.009011915759238062, 0...","[0.3645132923299154, -0.22668855509127542, 0.0...","[0.44498980088785645, -0.21377671142306073, -0...","[0.2028996048643067, -0.07039791359665602, -0....",1268.000000,4,-1,-1
82329,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,36.0,5,14.0,15.0,30.0,12.0,[],[],...,3,"[0.25056830107660755, 0.1855862403422956, -0.0...","[0.22000723570438951, -0.03295513889781696, 0....","[0.1900283816382599, -0.101841367221085, 0.037...","[0.1849982062512915, -0.038574753114996224, -0...","[0.2731820444500037, -0.09003645686019562, -0....",1583.333333,3,120,120
82330,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,27.0,30,5.0,15.0,30.0,14.0,[],[],...,0,"[0.4501867000452721, -0.2863792718427415, 0.24...","[0.153610738681069, 0.7683157290699305, -0.012...","[0.42336579128110136, 0.4415293631473783, 0.08...","[0.1645560370825661, 0.2974652276594942, -0.06...","[0.2136262495353285, 0.6889780980076565, -0.03...",487.000000,5,300,216
82331,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,43.0,1,16.0,30.0,50.0,13.0,[],[],...,3,"[0.0023119905200976844, 0.0012746774424303585,...","[0.06146393099923767, -0.005610505861232773, 0...","[0.002297610145839555, -0.0017582815189133496,...","[0.21403696108733744, -0.09024431543561566, -0...","[0.05431574129285235, -0.0031620523933319758, ...",968.666667,1,900,900


In [20]:

# 读取包含label的sample表
all_sample = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)

gen_path = '../generalization_data/cvjd_202208_6108.pkl'
gen_id = pd.read_pickle(gen_path)[['cv_id', 'jd_id', 'order_month']]

all_sample = all_sample.join(gen_id.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
all_sample

/Users/jinrirencai/opt/anaconda3/envs/tsinghua/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cv_id,jd_id,cv_code,jd_code,last_opt,da_label,rank_label,title,deadline,category_id,...,industry_jd2vec,industry_vec2,majorName_cut,majorName_jd2vec,majorName_vec2,skills_cut,skills_jd2vec,skills_vec2,ds,order_month
470,8a69c41582926ae60182a07b24c1048c,8a69f62f82926a8601829f1379a07648,C1008764509940281344,JD20220812101945714568,reject,2,2,多媒体算法工程师（视频算法）,NaN,100000,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['长春', '理工大学']","[0.3985162079334259, -0.9995342493057251, -0.0...","[0.11997296661138535, 0.26912960410118103, -0....","['Arm', ',', 'Git', ',', 'C++', ',', 'Demo', '...","[0.38910111784935, -0.09600549191236496, -0.16...","[0.03988659381866455, 0.09070292860269547, -0....",20220831,2022-08
471,8a69c41582926ae60182a0e09bf27664,8a69d77e82926b410182a083fbf602c0,C1008792400581292032,JD20220815155940455233,accept,3,3,采购工程师,NaN,250103,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['吉林大学'],"[0.09345181286334991, 0.009083555079996586, 0....","[0.23355810344219208, 0.3138801157474518, -0.1...","['Pc', ',', '篮球', ',', '行政', '管理']","[0.5843684673309326, 0.09454917907714844, 0.02...","[0.39833325147628784, 0.178728848695755, -0.74...",20220831,2022-08
472,8a69c41582926ae60182a119de713808,8a69f68782590ecb01826326b5e94919,C1008808140025819136,JD20220803180057511310,post,-1,2,销售工程师,NaN,210100,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['黑龙江', '大学']","[0.5287548899650574, -0.42875587940216064, 0.1...","[0.9195342659950256, 0.8215621709823608, -0.00...","['优化', ',', '架构', ',', '运营', ',', '搭建', ',', '...","[0.5232886075973511, 0.05269351974129677, -0.0...","[0.4764364957809448, -0.18185757100582123, -0....",20220831,2022-08
473,8a69c41582a3cc240182b0354c0e7bad,2c92071581ebf3390181ec9e500755e4,C1009871210596724736,JD20220711173647069409,drop,1,1,产品总监,NaN,110302,...,"[0.06698619574308395, -0.19367165863513947, -0...","[1.3965752124786377, 0.07573889195919037, -1.1...","['安徽', '财经大学']","[0.530929446220398, -0.6542360782623291, 0.157...","[0.24919170141220093, 0.20309053361415863, -0....","['#', 'null']","[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",20220831,2022-08
474,8a69c4a3827cc886018284b9612d0f03,8a69f601823a92b0018243dcb9c45f7a,C1006811292427849728,JD20220728161155132235,link,-1,1,GOOGLE推广,NaN,140104,...,"[0.3468320667743683, -0.4120320677757263, -0.0...","[0.7798252105712891, -0.039847563952207565, -0...",['湖南师范大学'],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","['Crm', ',', '电商', ',', '营销', ',', '策划', ',', ...","[0.5233139991760254, -0.056362878531217575, -0...","[0.47880709171295166, -0.09375546127557755, -0...",20220831,2022-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82316,8a69d71b82add4e10182aebff7d016d2,8a69c468826ee9b4018290a64a0c01bb,C1001546295099396096,JD20220812140313222488,reject,2,2,营销总监-服务中心,NaN,200101,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['中山大学'],"[0.0773172527551651, -0.2358638048171997, 0.00...","[-0.10793008655309677, 0.5528307557106018, 0.1...","['优化', ',', '策划', ',', '营销', ',', '团队', '建设', ...","[0.6373644471168518, 0.0508057177066803, 0.004...","[0.5669321417808533, -0.2156238704919815, -0.3...",20220831,2022-08
82325,8a69d7b4826d7dfb01826d7efad200f1,2c920715818a462a01818e3f9c0a2337,C972256921547575656,JD20220615174226328653,accept,3,3,Android-专家及leader以上,NaN,100202,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['University', ' ', 'of', ' ', 'Illinois']","[0.08156125992536545, 0.31228697299957275, 0.1...","[-0.15771661698818207, 0.5672192573547363, -0....","['Js', ',', 'C#', ',', 'Web', ',', 'CSS', ',',...","[0.3464146852493286, 0.07

In [21]:
# 拼接特征和label并保存
label_df = all_sample[['cv_id', 'jd_id', 'da_label', 'rank_label']]
raw_ext_new_feature_label = raw_ext_new_feature.join(label_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature_label.to_pickle('../generalization_data/gen_raw_ext_new_feature_label_6108.pkl')

In [22]:
raw_ext_new_feature_label

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,gen_position_tfidf_lsa,gen_requirement_tfidf_lsa,gen_skills_tfidf_lsa,gen_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label
90,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,41.0,1,19.0,60.0,70.0,12.0,"['211', '双一流']",[],...,"[0.01410938306399229, -0.0013842289481747908, ...","[0.16147995305672858, -0.07191881877515002, -0...","[0.2948113481706559, -0.1819010648461708, 0.01...","[0.4002958143776276, -0.18420725834968477, -0....",1122.000000,3,-1,800,0,0
183,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,33.0,30,8.0,15.0,30.0,14.0,[],[],...,"[0.1833094661547266, 0.5715641770033284, -0.00...","[0.42336579128110136, 0.4415293631473783, 0.08...","[0.3123019409501142, 0.25026041462341286, -0.0...","[0.2136262495353285, 0.6889780980076565, -0.03...",718.800000,5,-1,-1,1,1
312,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,33.0,1,9.0,30.0,45.0,16.0,"['211', '双一流']",['211'],...,"[0.1242462692048715, 0.059776515919823886, 0.0...","[0.13464711322627812, 0.03601774564762096, 0.0...","[0.2592719578121452, 0.19863144276159853, -0.1...","[0.0846159262961945, 0.14089215219884776, -0.0...",704.714286,2,-1,528,2,2
313,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,22.0,10,1.0,15.0,20.0,12.0,[],[],...,"[0.040317751879249734, -0.004257781078691267, ...","[0.2017619187252856, -0.1156944482993234, 0.12...","[0.20906464927123727, -0.16293904571516468, 0....","[0.10444744556880664, -0.03406922047311521, -0...",46.000000,4,108,72,1,1
314,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,22.0,40,1.0,10.0,15.0,12.0,[],[],...,"[0.040317751879249734, -0.004257781078691267, ...","[0.19550223541803413, -0.1128925577464503, 0.1...","[0.20906464927123727, -0.16293904571516468, 0....","[0.11715741531242865, -0.03963572163058257, -0...",46.000000,3,108,72,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,31.0,1,11.0,30.0,40.0,12.0,[],[],...,"[0.09276561511018605, -0.009011915759238062, 0...","[0.3645132923299154, -0.22668855509127542, 0.0...","[0.44498980088785645, -0.21377671142306073, -0...","[0.2028996048643067, -0.07039791359665602, -0....",1268.000000,4,-1,-1,3,3
82329,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,36.0,5,14.0,15.0,30.0,12.0,[],[],...,"[0.22000723570438951, -0.03295513889781696, 0....","[0.1900283816382599, -0.101841367221085, 0.037...","[0.1849982062512915, -0.038574753114996224, -0...","[0.2731820444500037, -0.09003645686019562, -0....",1583.333333,3,120,120,3,3
82330,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,27.0,30,5.0,15.0,30.0,14.0,[],[],...,"[0.153610738681069, 0.7683157290699305, -0.012...","[0.42336579128110136, 0.4415293631473783, 0.08...","[0.1645560370825661, 0.2974652276594942, -0.06...","[0.2136262495353285, 0.6889780980076565, -0.03...",487.000000,5,300,216,1,1
82331,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,43.0,1,16.0,30.0,50.0,13.0,[],[],...,"[0.06146393099923767, -0.005610505861232773, 0...","[0.002297610145839555, -0.0017582815189133496,...","[0.21403696108733744, -0.09024431543561566, -0...","[0.05431574129285235, -0.0031620523933319758, ...",968.666667,1,900,900,-1,2


In [26]:
sample_gen = raw_ext_new_feature_label

In [27]:
sample_gen.columns

Index(['cv_id', 'jd_id', 'age', 'recruit_number', 'workYear',
       'min_month_salary', 'max_month_salary', 'pay_months',
       'school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv',
       'location_range_jd', 'degree_index_cv', 'degree_index_jd',
       'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv',
       'gen_description_tfidf_lsa', 'gen_position_tfidf_lsa',
       'gen_requirement_tfidf_lsa', 'gen_skills_tfidf_lsa',
       'gen_title_category_tags_tfidf_lsa', 'work_duration_mean', 'equal_job',
       'parsed_desiredSalary', 'parsed_currentSalary', 'da_label',
       'rank_label'],
      dtype='object')

In [28]:
sample_gen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6108 entries, 90 to 82332
Data columns (total 28 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   cv_id                              6108 non-null   object 
 1   jd_id                              6108 non-null   object 
 2   age                                5579 non-null   float64
 3   recruit_number                     6108 non-null   int64  
 4   workYear                           5568 non-null   float64
 5   min_month_salary                   6060 non-null   float64
 6   max_month_salary                   6060 non-null   float64
 7   pay_months                         6060 non-null   float64
 8   school_labels_cv                   6108 non-null   object 
 9   school_labels_jd                   6108 non-null   object 
 10  expectlocation_range_cv            6108 non-null   object 
 11  location_range_jd                  6108 non-null   obj

In [25]:
!pwd

/Users/jinrirencai/Desktop/golden_data_from_github/generalization


In [29]:
# 中位数填充
import pickle

imp_pkl_filename = 'age_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_gen['age'] = imp.transform(sample_gen[['age']])

imp_pkl_filename = 'workYear_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_gen['workYear'] = imp.transform(sample_gen[['workYear']])

imp_pkl_filename = 'min_month_salary_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_gen['min_month_salary'] = imp.transform(sample_gen[['min_month_salary']])

imp_pkl_filename = 'max_month_salary_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_gen['max_month_salary'] = imp.transform(sample_gen[['max_month_salary']])

imp_pkl_filename = 'pay_months_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_gen['pay_months'] = imp.transform(sample_gen[['pay_months']])

imp_pkl_filename = 'work_duration_mean_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_gen['work_duration_mean'] = imp.transform(sample_gen[['work_duration_mean']])





# 这两个只能选一个接入模型
imp_pkl_filename = 'parsed_desiredSalary_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_gen['parsed_desiredSalary'] = imp.transform(sample_gen[['parsed_desiredSalary']])

imp_pkl_filename = 'parsed_currentSalary_missing_model.pkl'
with open(imp_pkl_filename, 'rb') as file:
    imp = pickle.load(file)
sample_gen['parsed_currentSalary'] = imp.transform(sample_gen[['parsed_currentSalary']])


sample_gen


,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,gen_position_tfidf_lsa,gen_requirement_tfidf_lsa,gen_skills_tfidf_lsa,gen_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label
90,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,41.0,1,19.0,60.0,70.0,12.0,"['211', '双一流']",[],...,"[0.01410938306399229, -0.0013842289481747908, ...","[0.16147995305672858, -0.07191881877515002, -0...","[0.2948113481706559, -0.1819010648461708, 0.01...","[0.4002958143776276, -0.18420725834968477, -0....",1122.000000,3,360.0,800.0,0,0
183,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,33.0,30,8.0,15.0,30.0,14.0,[],[],...,"[0.1833094661547266, 0.5715641770033284, -0.00...","[0.42336579128110136, 0.4415293631473783, 0.08...","[0.3123019409501142, 0.25026041462341286, -0.0...","[0.2136262495353285, 0.6889780980076565, -0.03...",718.800000,5,360.0,300.0,1,1
312,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,33.0,1,9.0,30.0,45.0,16.0,"['211', '双一流']",['211'],...,"[0.1242462692048715, 0.059776515919823886, 0.0...","[0.13464711322627812, 0.03601774564762096, 0.0...","[0.2592719578121452, 0.19863144276159853, -0.1...","[0.0846159262961945, 0.14089215219884776, -0.0...",704.714286,2,360.0,528.0,2,2
313,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,22.0,10,1.0,15.0,20.0,12.0,[],[],...,"[0.040317751879249734, -0.004257781078691267, ...","[0.2017619187252856, -0.1156944482993234, 0.12...","[0.20906464927123727, -0.16293904571516468, 0....","[0.10444744556880664, -0.03406922047311521, -0...",46.000000,4,108.0,72.0,1,1
314,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,22.0,40,1.0,10.0,15.0,12.0,[],[],...,"[0.040317751879249734, -0.004257781078691267, ...","[0.19550223541803413, -0.1128925577464503, 0.1...","[0.20906464927123727, -0.16293904571516468, 0....","[0.11715741531242865, -0.03963572163058257, -0...",46.000000,3,108.0,72.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,31.0,1,11.0,30.0,40.0,12.0,[],[],...,"[0.09276561511018605, -0.009011915759238062, 0...","[0.3645132923299154, -0.22668855509127542, 0.0...","[0.44498980088785645, -0.21377671142306073, -0...","[0.2028996048643067, -0.07039791359665602, -0....",1268.000000,4,360.0,300.0,3,3
82329,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,36.0,5,14.0,15.0,30.0,12.0,[],[],...,"[0.22000723570438951, -0.03295513889781696, 0....","[0.1900283816382599, -0.101841367221085, 0.037...","[0.1849982062512915, -0.038574753114996224, -0...","[0.2731820444500037, -0.09003645686019562, -0....",1583.333333,3,120.0,120.0,3,3
82330,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,27.0,30,5.0,15.0,30.0,14.0,[],[],...,"[0.153610738681069, 0.7683157290699305, -0.012...","[0.42336579128110136, 0.4415293631473783, 0.08...","[0.1645560370825661, 0.2974652276594942, -0.06...","[0.2136262495353285, 0.6889780980076565, -0.03...",487.000000,5,300.0,216.0,1,1
82331,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,43.0,1,16.0,30.0,50.0,13.0,[],[],...,"[0.06146393099923767, -0.005610505861232773, 0...","[0.002297610145839555, -0.0017582815189133496,...","[0.21403696108733744, -0.09024431543561566, -0...","[0.05431574129285235, -0.0031620523933319758, ...",968.666667,1,900.0,900.0,-1,2


In [30]:
sample_gen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6108 entries, 90 to 82332
Data columns (total 28 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   cv_id                              6108 non-null   object 
 1   jd_id                              6108 non-null   object 
 2   age                                6108 non-null   float64
 3   recruit_number                     6108 non-null   int64  
 4   workYear                           6108 non-null   float64
 5   min_month_salary                   6108 non-null   float64
 6   max_month_salary                   6108 non-null   float64
 7   pay_months                         6108 non-null   float64
 8   school_labels_cv                   6108 non-null   object 
 9   school_labels_jd                   6108 non-null   object 
 10  expectlocation_range_cv            6108 non-null   object 
 11  location_range_jd                  6108 non-null   obj

# 手动cross

In [31]:
# 首先把school_labels_cv和school_labels_jd转成列表
def school_lables_to_list(series, school_col):
    school_labels = series[school_col]
    school_labels_list = [item.strip('\'') for item in school_labels.strip('[]').replace(' ', '').split(',')]
    if school_labels_list[0] == '':
        return []
    return school_labels_list
sample_gen['school_labels_cv_list'] = sample_gen.apply(school_lables_to_list, axis=1, args=('school_labels_cv', ))
sample_gen['school_labels_jd_list'] = sample_gen.apply(school_lables_to_list, axis=1, args=('school_labels_jd', ))

sample_gen

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,gen_skills_tfidf_lsa,gen_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,school_labels_cv_list,school_labels_jd_list
90,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,41.0,1,19.0,60.0,70.0,12.0,"['211', '双一流']",[],...,"[0.2948113481706559, -0.1819010648461708, 0.01...","[0.4002958143776276, -0.18420725834968477, -0....",1122.000000,3,360.0,800.0,0,0,"[211, 双一流]",[]
183,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,33.0,30,8.0,15.0,30.0,14.0,[],[],...,"[0.3123019409501142, 0.25026041462341286, -0.0...","[0.2136262495353285, 0.6889780980076565, -0.03...",718.800000,5,360.0,300.0,1,1,[],[]
312,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,33.0,1,9.0,30.0,45.0,16.0,"['211', '双一流']",['211'],...,"[0.2592719578121452, 0.19863144276159853, -0.1...","[0.0846159262961945, 0.14089215219884776, -0.0...",704.714286,2,360.0,528.0,2,2,"[211, 双一流]",[211]
313,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,22.0,10,1.0,15.0,20.0,12.0,[],[],...,"[0.20906464927123727, -0.16293904571516468, 0....","[0.10444744556880664, -0.03406922047311521, -0...",46.000000,4,108.0,72.0,1,1,[],[]
314,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,22.0,40,1.0,10.0,15.0,12.0,[],[],...,"[0.20906464927123727, -0.16293904571516468, 0....","[0.11715741531242865, -0.03963572163058257, -0...",46.000000,3,108.0,72.0,2,2,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,31.0,1,11.0,30.0,40.0,12.0,[],[],...,"[0.44498980088785645, -0.21377671142306073, -0...","[0.2028996048643067, -0.07039791359665602, -0....",1268.000000,4,360.0,300.0,3,3,[],[]
82329,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,36.0,5,14.0,15.0,30.0,12.0,[],[],...,"[0.1849982062512915, -0.038574753114996224, -0...","[0.2731820444500037, -0.09003645686019562, -0....",1583.333333,3,120.0,120.0,3,3,[],[]
82330,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,27.0,30,5.0,15.0,30.0,14.0,[],[],...,"[0.1645560370825661, 0.2974652276594942, -0.06...","[0.2136262495353285, 0.6889780980076565, -0.03...",487.000000,5,300.0,216.0,1,1,[],[]
82331,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,43.0,1,16.0,30.0,50.0,13.0,[],[],...,"[0.21403696108733744, -0.09024431543561566, -0...","[0.05431574129285235, -0.0031620523933319758, ...",968.666667,1,900.0,900.0,-1,2,[],[]


In [32]:
# school等级交叉
def school_intersect_cvjd(series, school_labels_cv, school_labels_jd):
    school_labels_cv = series[school_labels_cv]
    school_labels_jd = series[school_labels_jd]
    # jd没有学历要求or解析不出来
    if not school_labels_jd:
        return 1
    if set(school_labels_cv).intersection(set(school_labels_jd)):
        return 1
    return 0

sample_gen['school_intersect_cvjd'] = sample_gen.apply(school_intersect_cvjd, axis=1, args=('school_labels_cv_list', 'school_labels_jd_list'))
sample_gen

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,gen_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd
90,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,41.0,1,19.0,60.0,70.0,12.0,"['211', '双一流']",[],...,"[0.4002958143776276, -0.18420725834968477, -0....",1122.000000,3,360.0,800.0,0,0,"[211, 双一流]",[],1
183,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,33.0,30,8.0,15.0,30.0,14.0,[],[],...,"[0.2136262495353285, 0.6889780980076565, -0.03...",718.800000,5,360.0,300.0,1,1,[],[],1
312,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,33.0,1,9.0,30.0,45.0,16.0,"['211', '双一流']",['211'],...,"[0.0846159262961945, 0.14089215219884776, -0.0...",704.714286,2,360.0,528.0,2,2,"[211, 双一流]",[211],1
313,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,22.0,10,1.0,15.0,20.0,12.0,[],[],...,"[0.10444744556880664, -0.03406922047311521, -0...",46.000000,4,108.0,72.0,1,1,[],[],1
314,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,22.0,40,1.0,10.0,15.0,12.0,[],[],...,"[0.11715741531242865, -0.03963572163058257, -0...",46.000000,3,108.0,72.0,2,2,[],[],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,31.0,1,11.0,30.0,40.0,12.0,[],[],...,"[0.2028996048643067, -0.07039791359665602, -0....",1268.000000,4,360.0,300.0,3,3,[],[],1
82329,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,36.0,5,14.0,15.0,30.0,12.0,[],[],...,"[0.2731820444500037, -0.09003645686019562, -0....",1583.333333,3,120.0,120.0,3,3,[],[],1
82330,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,27.0,30,5.0,15.0,30.0,14.0,[],[],...,"[0.2136262495353285, 0.6889780980076565, -0.03...",487.000000,5,300.0,216.0,1,1,[],[],1
82331,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,43.0,1,16.0,30.0,50.0,13.0,[],[],...,"[0.05431574129285235, -0.0031620523933319758, ...",968.666667,1,900.0,900.0,-1,2,[],[],1


In [33]:
# 首先把expectlocation_range_cv和location_range_jd转成列表
def school_lables_to_list(series, location_col):
    location_labels = series[location_col]
    location_labels_list = [item.strip('\'') for item in location_labels.strip('[]').replace(' ', '').split(',')]
    if location_labels_list[0] == '':
        return []
    return location_labels_list

sample_gen['location_range_jd_list'] = sample_gen.apply(school_lables_to_list, axis=1, args=('location_range_jd', ))
sample_gen['expectlocation_range_cv_list'] = sample_gen.apply(school_lables_to_list, axis=1, args=('expectlocation_range_cv', ))

sample_gen

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list
90,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,41.0,1,19.0,60.0,70.0,12.0,"['211', '双一流']",[],...,3,360.0,800.0,0,0,"[211, 双一流]",[],1,[],[未知市]
183,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,33.0,30,8.0,15.0,30.0,14.0,[],[],...,5,360.0,300.0,1,1,[],[],1,[],[未知市]
312,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,33.0,1,9.0,30.0,45.0,16.0,"['211', '双一流']",['211'],...,2,360.0,528.0,2,2,"[211, 双一流]",[211],1,[],[南京市]
313,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,22.0,10,1.0,15.0,20.0,12.0,[],[],...,4,108.0,72.0,1,1,[],[],1,[],[上海市]
314,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,22.0,40,1.0,10.0,15.0,12.0,[],[],...,3,108.0,72.0,2,2,[],[],1,[],[上海市]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,31.0,1,11.0,30.0,40.0,12.0,[],[],...,4,360.0,300.0,3,3,[],[],1,[],[杭州市]
82329,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,36.0,5,14.0,15.0,30.0,12.0,[],[],...,3,120.0,120.0,3,3,[],[],1,[],[合肥市]
82330,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,27.0,30,5.0,15.0,30.0,14.0,[],[],...,5,300.0,216.0,1,1,[],[],1,[],[上海市]
82331,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,43.0,1,16.0,30.0,50.0,13.0,[],[],...,1,900.0,900.0,-1,2,[],[],1,[],"[上海市, 北京市]"


In [34]:
# location交叉
def location_intersect_cvjd(series, location_labels_cv, location_labels_jd):
    location_labels_cv = series[location_labels_cv]
    location_labels_jd = series[location_labels_jd]
    # 如果jd没有地域
    if not location_labels_jd:
        return 1
    if set(location_labels_cv).intersection(set(location_labels_jd)):
        return 1
    return 0

sample_gen['location_intersect_cvjd'] = sample_gen.apply(location_intersect_cvjd, axis=1, args=('expectlocation_range_cv_list', 'location_range_jd_list'))
sample_gen

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list,location_intersect_cvjd
90,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,41.0,1,19.0,60.0,70.0,12.0,"['211', '双一流']",[],...,360.0,800.0,0,0,"[211, 双一流]",[],1,[],[未知市],1
183,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,33.0,30,8.0,15.0,30.0,14.0,[],[],...,360.0,300.0,1,1,[],[],1,[],[未知市],1
312,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,33.0,1,9.0,30.0,45.0,16.0,"['211', '双一流']",['211'],...,360.0,528.0,2,2,"[211, 双一流]",[211],1,[],[南京市],1
313,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,22.0,10,1.0,15.0,20.0,12.0,[],[],...,108.0,72.0,1,1,[],[],1,[],[上海市],1
314,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,22.0,40,1.0,10.0,15.0,12.0,[],[],...,108.0,72.0,2,2,[],[],1,[],[上海市],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,31.0,1,11.0,30.0,40.0,12.0,[],[],...,360.0,300.0,3,3,[],[],1,[],[杭州市],1
82329,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,36.0,5,14.0,15.0,30.0,12.0,[],[],...,120.0,120.0,3,3,[],[],1,[],[合肥市],1
82330,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,27.0,30,5.0,15.0,30.0,14.0,[],[],...,300.0,216.0,1,1,[],[],1,[],[上海市],1
82331,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,43.0,1,16.0,30.0,50.0,13.0,[],[],...,900.0,900.0,-1,2,[],[],1,[],"[上海市, 北京市]",1


In [35]:
# degree交叉
def degree_match_cvjd(series, degree_index_cv, degree_index_jd):
    degree_index_cv = series['degree_index_cv']
    degree_index_jd = series['degree_index_jd']
    if degree_index_cv >= degree_index_jd:
        return 1
    else:
        return 0

sample_gen['degree_match_cvjd'] = sample_gen.apply(degree_match_cvjd, axis=1, args=('degree_index_cv', 'degree_index_jd'))
sample_gen

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,parsed_currentSalary,da_label,rank_label,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list,location_intersect_cvjd,degree_match_cvjd
90,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,41.0,1,19.0,60.0,70.0,12.0,"['211', '双一流']",[],...,800.0,0,0,"[211, 双一流]",[],1,[],[未知市],1,1
183,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,33.0,30,8.0,15.0,30.0,14.0,[],[],...,300.0,1,1,[],[],1,[],[未知市],1,0
312,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,33.0,1,9.0,30.0,45.0,16.0,"['211', '双一流']",['211'],...,528.0,2,2,"[211, 双一流]",[211],1,[],[南京市],1,1
313,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,22.0,10,1.0,15.0,20.0,12.0,[],[],...,72.0,1,1,[],[],1,[],[上海市],1,1
314,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,22.0,40,1.0,10.0,15.0,12.0,[],[],...,72.0,2,2,[],[],1,[],[上海市],1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,31.0,1,11.0,30.0,40.0,12.0,[],[],...,300.0,3,3,[],[],1,[],[杭州市],1,1
82329,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,36.0,5,14.0,15.0,30.0,12.0,[],[],...,120.0,3,3,[],[],1,[],[合肥市],1,1
82330,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,27.0,30,5.0,15.0,30.0,14.0,[],[],...,216.0,1,1,[],[],1,[],[上海市],1,1
82331,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,43.0,1,16.0,30.0,50.0,13.0,[],[],...,900.0,-1,2,[],[],1,[],"[上海市, 北京市]",1,1


In [36]:
sample_gen['rank_label'].value_counts()

1    2531
2    2516
3     891
0     170
Name: rank_label, dtype: int64

In [37]:
# 生成模型训练label
def model_label_gen(series, rank_label):
    rank_label = series[rank_label]
    if rank_label in [0, 1]:
        return 0
    return 1
sample_gen['model_label'] = sample_gen.apply(model_label_gen, axis=1, args=('rank_label', ))
sample_gen

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,da_label,rank_label,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list,location_intersect_cvjd,degree_match_cvjd,model_label
90,2c9207157f8e60cd017f960e491b6f43,8a69c468826ee9b401828badfc6c3158,41.0,1,19.0,60.0,70.0,12.0,"['211', '双一流']",[],...,0,0,"[211, 双一流]",[],1,[],[未知市],1,1,0
183,2c9207158059e82b018093a6e4cb418b,2c92071580dd3ab70180f18714e119c0,33.0,30,8.0,15.0,30.0,14.0,[],[],...,1,1,[],[],1,[],[未知市],1,0,0
312,8a69c40a8256f56001825ca41faf64ef,8a69f68e82198e9101821a368c682309,33.0,1,9.0,30.0,45.0,16.0,"['211', '双一流']",['211'],...,2,2,"[211, 双一流]",[211],1,[],[南京市],1,1,1
313,8a69c40a8256f56001826286256928f3,8a69c47b82590db9018266992c3969ee,22.0,10,1.0,15.0,20.0,12.0,[],[],...,1,1,[],[],1,[],[上海市],1,1,0
314,8a69c40a8256f56001826286256928f3,8a69c4a8820f0ee001821446eaaf33e6,22.0,40,1.0,10.0,15.0,12.0,[],[],...,2,2,[],[],1,[],[上海市],1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82328,8a69f60582c8626e0182c9fc367050e6,8a69c4df82b081d30182c391c1f51bfc,31.0,1,11.0,30.0,40.0,12.0,[],[],...,3,3,[],[],1,[],[杭州市],1,1,1
82329,8a69f60582c8626e0182cedb7ca86dc8,8a69c47b82b07fdc0182b3dcd51b7a71,36.0,5,14.0,15.0,30.0,12.0,[],[],...,3,3,[],[],1,[],[合肥市],1,1,1
82330,8a69f61382b080760182c37a984f3a9a,2c92071580dd3ab70180f18714e119c0,27.0,30,5.0,15.0,30.0,14.0,[],[],...,1,1,[],[],1,[],[上海市],1,1,0
82331,8a69f64782e59cc90182f37791dc6649,8a69d77e82926b410182b011869b3b74,43.0,1,16.0,30.0,50.0,13.0,[],[],...,-1,2,[],[],1,[],"[上海市, 北京市]",1,1,1


In [38]:
sample_gen['model_label'].value_counts()

1    3407
0    2701
Name: model_label, dtype: int64

In [39]:
# 将jd中的月薪转成年薪
def multiply_cols_fun(series, col1, col2):
    col1_value = series[col1]
    col2_value = series[col2]
    return col1_value * col2_value

sample_gen['min_annual_salary'] = sample_gen.apply(multiply_cols_fun, axis=1, args=('min_month_salary', 'pay_months'))
sample_gen['max_annual_salary'] = sample_gen.apply(multiply_cols_fun, axis=1, args=('max_month_salary', 'pay_months'))

In [40]:
# 先把workyear_range_jd转成列表
def school_lables_to_list(series, location_col):
    location_labels = series[location_col]
    location_labels_list = [item.strip('\'') for item in location_labels.strip('[]').replace(' ', '').split(',')]
    first = int(location_labels_list[0])
    second = int(location_labels_list[1])
    return [first, second]

sample_gen['workyear_range_jd_list'] = sample_gen.apply(school_lables_to_list, axis=1, args=('workyear_range_jd', ))

sample_gen[['workyear_range_jd', 'workyear_range_jd_list']]

,workyear_range_jd,workyear_range_jd_list
90,"[5, 10]","[5, 10]"
183,"[3, 5]","[3, 5]"
312,"[5, 10]","[5, 10]"
313,"[0, 1]","[0, 1]"
314,"[1, 3]","[1, 3]"
...,...,...
82328,"[5, 100]","[5, 100]"
82329,"[5, 10]","[5, 10]"
82330,"[3, 5]","[3, 5]"
82331,"[3, 5]","[3, 5]"


In [41]:
# work year match
def workYear_match_fun(series, workyear_range_jd_list, workYear):
    workyear_range = series[workyear_range_jd_list]
    workYear = series[workYear]
    if workyear_range[0] <= workYear <= workyear_range[1]:
        return 1
    else:
        return 0
    
sample_gen['workyear_match_cvjd'] = sample_gen.apply(workYear_match_fun, axis=1, args=('workyear_range_jd_list', 'workYear'))

In [42]:
# 好学校
def keyU_label_fun(series, school_labels_cv_list):
    school_labels = series[school_labels_cv_list]
    if school_labels:
        return 1
    else:
        return 0
sample_gen['keyU'] = sample_gen.apply(keyU_label_fun, axis=1, args=('school_labels_cv_list',))

In [45]:
sample_gen['gen_description_tfidf_lsa'][90].__class__

list

In [46]:
sample_gen.columns

Index(['cv_id', 'jd_id', 'age', 'recruit_number', 'workYear',
       'min_month_salary', 'max_month_salary', 'pay_months',
       'school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv',
       'location_range_jd', 'degree_index_cv', 'degree_index_jd',
       'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv',
       'gen_description_tfidf_lsa', 'gen_position_tfidf_lsa',
       'gen_requirement_tfidf_lsa', 'gen_skills_tfidf_lsa',
       'gen_title_category_tags_tfidf_lsa', 'work_duration_mean', 'equal_job',
       'parsed_desiredSalary', 'parsed_currentSalary', 'da_label',
       'rank_label', 'school_labels_cv_list', 'school_labels_jd_list',
       'school_intersect_cvjd', 'location_range_jd_list',
       'expectlocation_range_cv_list', 'location_intersect_cvjd',
       'degree_match_cvjd', 'model_label', 'min_annual_salary',
       'max_annual_salary', 'workyear_range_jd_list', 'workyear_match_cvjd',
       'keyU'],
      dtype='object')

In [47]:
# sample train的几个tfidf列重命名
sample_test = sample_gen.rename(columns={
                    'gen_description_tfidf_lsa': 'description_tfidf_pca',
                    'gen_position_tfidf_lsa': 'position_tfidf_pca',
                    'gen_requirement_tfidf_lsa': 'requirement_tfidf_pca',
                    'gen_skills_tfidf_lsa': 'kills_tfidf_pca',
                    'gen_title_category_tags_tfidf_lsa': 'title_category_tags_tfidf_pca' 
})
sample_test.columns

Index(['cv_id', 'jd_id', 'age', 'recruit_number', 'workYear',
       'min_month_salary', 'max_month_salary', 'pay_months',
       'school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv',
       'location_range_jd', 'degree_index_cv', 'degree_index_jd',
       'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv',
       'description_tfidf_pca', 'position_tfidf_pca', 'requirement_tfidf_pca',
       'kills_tfidf_pca', 'title_category_tags_tfidf_pca',
       'work_duration_mean', 'equal_job', 'parsed_desiredSalary',
       'parsed_currentSalary', 'da_label', 'rank_label',
       'school_labels_cv_list', 'school_labels_jd_list',
       'school_intersect_cvjd', 'location_range_jd_list',
       'expectlocation_range_cv_list', 'location_intersect_cvjd',
       'degree_match_cvjd', 'model_label', 'min_annual_salary',
       'max_annual_salary', 'workyear_range_jd_list', 'workyear_match_cvjd',
       'keyU'],
      dtype='object')

In [48]:
# 几个list如何处理
# parsed_currentSalary特征不使用
# min_month_salary和max_month_salary分别乘以pay_months，得到年薪数据，以k为单位。已处理
# salary特征可信度不高，清华模型就是简单的输入deep侧，除此以外没有任何处理。
COLUMNS = ['age', 'recruit_number', 'workYear', 'min_annual_salary', 'max_annual_salary', 
            'degree_index_cv', 'degree_index_jd', 'gender_index_cv', 'job_intentstatus_cv',
            'description_tfidf_pca', 'position_tfidf_pca', 'requirement_tfidf_pca', 'kills_tfidf_pca', 'title_category_tags_tfidf_pca',
            'work_duration_mean', 'equal_job', 'parsed_desiredSalary', 'school_intersect_cvjd', 'location_intersect_cvjd',
            'degree_match_cvjd', 'workyear_match_cvjd', 'keyU']

LABEL_COLUMN = "model_label"

# 离散特征
CATEGORICAL_COLUMNS = ['degree_index_cv', 'degree_index_jd', 'gender_index_cv', 'job_intentstatus_cv', 'school_intersect_cvjd',
                        'location_intersect_cvjd', 'degree_match_cvjd', 'workyear_match_cvjd', 'keyU']

# 文本向量特征
ARRAY_COLUMNS = ['description_tfidf_pca', 'position_tfidf_pca', 'requirement_tfidf_pca', 'kills_tfidf_pca', 'title_category_tags_tfidf_pca']

# 连续特征
CONTINUOUS_COLUMNS = ['age', 'recruit_number', 'workYear', 'min_annual_salary', 'max_annual_salary', 
                    'work_duration_mean', 'equal_job', 'parsed_desiredSalary', ]


In [49]:
COLUMNS.sort()
print(COLUMNS)
print(len(COLUMNS))

['age', 'degree_index_cv', 'degree_index_jd', 'degree_match_cvjd', 'description_tfidf_pca', 'equal_job', 'gender_index_cv', 'job_intentstatus_cv', 'keyU', 'kills_tfidf_pca', 'location_intersect_cvjd', 'max_annual_salary', 'min_annual_salary', 'parsed_desiredSalary', 'position_tfidf_pca', 'recruit_number', 'requirement_tfidf_pca', 'school_intersect_cvjd', 'title_category_tags_tfidf_pca', 'workYear', 'work_duration_mean', 'workyear_match_cvjd']
22


In [50]:
a = CATEGORICAL_COLUMNS + ARRAY_COLUMNS + CONTINUOUS_COLUMNS
a.sort()
print(a)
print(len(a))

['age', 'degree_index_cv', 'degree_index_jd', 'degree_match_cvjd', 'description_tfidf_pca', 'equal_job', 'gender_index_cv', 'job_intentstatus_cv', 'keyU', 'kills_tfidf_pca', 'location_intersect_cvjd', 'max_annual_salary', 'min_annual_salary', 'parsed_desiredSalary', 'position_tfidf_pca', 'recruit_number', 'requirement_tfidf_pca', 'school_intersect_cvjd', 'title_category_tags_tfidf_pca', 'workYear', 'work_duration_mean', 'workyear_match_cvjd']
22


In [51]:
all_columns = ['cv_id', 'jd_id', 'model_label'] + COLUMNS
print(all_columns)
print(len(all_columns))

['cv_id', 'jd_id', 'model_label', 'age', 'degree_index_cv', 'degree_index_jd', 'degree_match_cvjd', 'description_tfidf_pca', 'equal_job', 'gender_index_cv', 'job_intentstatus_cv', 'keyU', 'kills_tfidf_pca', 'location_intersect_cvjd', 'max_annual_salary', 'min_annual_salary', 'parsed_desiredSalary', 'position_tfidf_pca', 'recruit_number', 'requirement_tfidf_pca', 'school_intersect_cvjd', 'title_category_tags_tfidf_pca', 'workYear', 'work_duration_mean', 'workyear_match_cvjd']
25


In [52]:
len(sample_test)

6108

In [53]:
sample_test[all_columns].to_pickle('../generalization_data/sample_gen_processed_processed_6108.pkl')